In [1]:
import pandas as pd
import cobra

In [2]:
df_in = pd.read_excel('./input/rxn_details.xlsx')
df_in.index = df_in.rxn.to_list()

# Fix pathway assignment
df_in.loc['SUCACCT_m', 'subsystem'] = 'Citric acid cycle'
df_in.loc['PYRDC2_c', 'subsystem'] = 'Unassigned'
df_in.loc['PPM_c', 'subsystem'] = 'Alternative carbon metabolism'
df_in.loc['MALS_x', 'subsystem'] = 'Unassigned'
df_in.loc['ACOAHi_m', 'subsystem'] = 'Unassigned'
df_in.loc['ATPM_c', 'subsystem'] = 'Pseudoreaction'
rxns_new_pseudo = ['DICAT'+str(i)+'_c' for i in range(1,9)] + \
                ['HEME'+str(i)+'_c' for i in range(1,5)] + \
                ['MONCAT'+str(i)+'_c' for i in range(1,4)]
for rxn in rxns_new_pseudo:
    df_in.loc[rxn, 'subsystem'] = 'Pseudoreaction'

In [3]:
rxns = []
for i in df_in.index:
    if df_in.subsystem[i] not in ['Transport', 'Exchange', 'Pseudoreaction']:
        rxns.append(i)

In [4]:
len(rxns)

1104

In [5]:
model = cobra.io.load_json_model('../build_model/input/GSM_iSace1144_rba.json')
model.solver = 'cplex'

# Valid turnoff (cycling)
model.reactions.LACPYRt_c_m.knock_out()

# Turn off due to regulation
# Ethanol utilization enzyme expression is repressed under glucose-utilizing condition (pmid:3290644)
model.reactions.ALCD2i1_c.knock_out()
# Ethanol-acetaldehyde shuttle only active under anaerobic condition (pmid:10940011)
model.reactions.ALCD2i2_m.knock_out()
# Totally unrelated reactions that form transhydrogenase cycle
model.reactions.get_by_id('4ABTORy_c').knock_out()
model.reactions.get_by_id('4ABTORx_c').knock_out()
# Disable FADH2 transport across mitochondrial membrane. Enable this creating fumarate-succinate shuttle that enables proton gradient generation
model.reactions.get_by_id('FADH2t_c_m').knock_out()
model.reactions.get_by_id('ICDHy_c').knock_out()
# Disable glycolysis bypass via methylglyoxal (disable lactate transport to mito)
model.reactions.get_by_id('LACtps_m').knock_out()

# Turn off alternative unparameterized pathways (which is cheaper to operate since unparameterized pathways used median kapp)
model.reactions.get_by_id('G3PAT_l').knock_out()
model.reactions.get_by_id('AGPAT_l').knock_out()
model.reactions.get_by_id('PAP_vm').knock_out()
model.reactions.get_by_id('PAP_gm').knock_out()
model.reactions.get_by_id('DHAPAT_rm').knock_out()
model.reactions.get_by_id('DHAPAT_l').knock_out()
model.reactions.get_by_id('G3PD1i_m').knock_out()

# Turn off cytosolic aconitase due to formation of thermo-infeasible cycle
model.reactions.ACONTa_c.knock_out()
model.reactions.ACONTb_c.knock_out()

# Fix HCO3t_c_n reaction reversibility
model.reactions.HCO3t_c_n.bounds = (-1000,1000)

# YNB media
model.reactions.get_by_id('EX_thm_e').lower_bound = -1000
model.reactions.get_by_id('EX_ribflv_e').lower_bound = -1000
model.reactions.get_by_id('EX_nac_e').lower_bound = -1000
model.reactions.get_by_id('EX_pydxn_e').lower_bound = -1000
model.reactions.get_by_id('EX_fol_e').lower_bound = -1000
model.reactions.get_by_id('EX_pnto__R_e').lower_bound = -1000
model.reactions.get_by_id('EX_4abz_e').lower_bound = -1000
model.reactions.get_by_id('EX_inost_e').lower_bound = -1000

Using license file /home/hvdinh16/Workspace/Softwares/gurobi910/linux64/gurobi.lic
Academic license - for non-commercial use only - expires 2023-07-30


In [7]:
rxns = [rxn.id for rxn in model.reactions]
with model as m:
    # Uptake rate increase slightly by 1e-2 to compensate for uptake relaxation (resolve numerical issue) in RBA run
    m.reactions.EX_glc__D_e.bounds = (-13.21, -13.21)
    m.reactions.BIOMASS_AERO_SC_hvd.bounds = (0.4236, 0.4236)
    m.reactions.ATPM_c.lower_bound = 1
    
    m.objective = dict()
    fva = cobra.flux_analysis.flux_variability_analysis(m, reaction_list=rxns)

In [8]:
rxns = df_in.index.to_list()

In [9]:
cols =['rxn', 'name', 'subsystem', 'rba_min', 'rba_max', 'fva_min', 'fva_max']
df_res = pd.DataFrame(columns=cols, index=rxns)

df_res['rxn'] = df_in['rxn']
df_res['name'] = df_in['name']
df_res['subsystem'] = df_in['subsystem']

for rxn in df_res.index:
    df_res.loc[rxn, 'fva_min'] = fva.minimum[rxn]
    df_res.loc[rxn, 'fva_max'] = fva.maximum[rxn]

In [10]:
with open('./vary_flux_fragments_minflux/minflux.txt') as f:
    text = f.read().split('\n')[:-1]
rbamin = dict()
for line in text:
    _,rxn,stat,val = line.split('\t')
    if stat == 'optimal':
        rbamin[rxn] = float(val)
    else:
        print('Something wrong with ' + rxn + ', min run, stat = ' + stat)
        
with open('./vary_flux_fragments_maxflux/maxflux.txt') as f:
    text = f.read().split('\n')[:-1]
rbamax = dict()
for line in text:
    _,rxn,stat,val = line.split('\t')
    if stat == 'optimal':
        rbamax[rxn] = float(val)
    else:
        print('Something wrong with ' + rxn + ', max run, stat = ' + stat)
        
for rxn in df_res.index:
    df_res.loc[rxn, 'rba_min'] = rbamin[rxn]
    df_res.loc[rxn, 'rba_max'] = rbamax[rxn]

### Process for reversibility

In [11]:
# Remove pseudoreaction
df_res = df_res[df_res.subsystem != 'Pseudoreaction']
df_res = df_res[df_res.subsystem != 'Transport']
df_res = df_res[df_res.subsystem != 'Exchange']

# Handle reversible rxns with non-zero reverse flux
df_res = df_res[(df_res.fva_max > 0) | (df_res.fva_min < 0)]

idx_remove = []
for rxn in df_res.index:
    fvamin = df_res.fva_min[rxn]
    fvamax = df_res.fva_max[rxn]
    rbamin = df_res.rba_min[rxn]
    rbamax = df_res.rba_max[rxn]
    
    rfwd = rxn + '_FWD'
    rrev = rxn + '_REV'
    if fvamin < 0:
        idx_remove.append(rxn)
        
        df_res.loc[rrev, 'fva_min'] = 0
        df_res.loc[rrev, 'fva_max'] = abs(fvamin)
        df_res.loc[rrev, 'rba_min'] = 0
        df_res.loc[rrev, 'rba_max'] = abs(rbamin)
        df_res.loc[rrev, 'rxn'] = rrev
        df_res.loc[rrev, 'name'] = df_res.name[rxn]
        df_res.loc[rrev, 'subsystem'] = df_res.subsystem[rxn]
        
        if fvamax > 0:
            df_res.loc[rfwd, 'fva_min'] = 0
            df_res.loc[rfwd, 'fva_max'] = fvamax
            df_res.loc[rfwd, 'rba_min'] = 0
            df_res.loc[rfwd, 'rba_max'] = rbamax
            df_res.loc[rfwd, 'rxn'] = rfwd
            df_res.loc[rfwd, 'name'] = df_res.name[rxn]
            df_res.loc[rfwd, 'subsystem'] = df_res.subsystem[rxn]

In [12]:
# Save processed FVA RBA results
idx_remove_checked = [i for i in idx_remove if i in df_res.index]
df_res = df_res.drop(index=idx_remove_checked, axis=0)
df_res.to_excel('./results/FVA_RBA_bounds_processed_for_reversibility.xlsx', index=None)